#### Fix Relative paths for the notebook

In [1]:
import sys
import os

# Get the absolute path of the project root (adjust as needed)
project_root = os.path.abspath("..")  # Moves one level up to project root

# Add to sys.path if not already present
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [3]:
from database.database import get_dtc_with_components

export_template_path: /root/fast-api-graph/outflow/templates
qdrant_host: localhost
qdrant_port: 6333
neo4j_connection: bolt://localhost:7687
neo4j_user: neo4j
neo4j_password: ********
Neo4j Connection Successful


In [2]:
import os
from xmltodict import parse

file_content = None
# calculate the hash of the file
with open(f"../data/input/io_lists/PtIOList_COO.xml", "rb") as file:
    file_content = file.read()
    # parse the xml file content to a dictionary
    pt_io_list = parse(file_content)
    io_list = []
    if (
        "IO" in pt_io_list["PtIOList"]
        and type(pt_io_list["PtIOList"]["IO"]) == list
    ):
        for io in pt_io_list["PtIOList"]["IO"]:
            io_list.append(io)
    elif "IO" in pt_io_list["PtIOList"]:
        io_list.append(pt_io_list["PtIOList"]["IO"])

    print(f"Processing IO Mapping - total IOs: {len(io_list)}")

Processing IO Mapping - total IOs: 658


In [3]:
import importlib
import graphs.io_processor  # Ensure the module is imported first
importlib.reload(graphs.io_processor)

# Now reassign the imported attribute
io_processor = graphs.io_processor.graph

export_template_path: /root/fast-api-graph/outflow/templates
qdrant_host: localhost
qdrant_port: 6333
neo4j_connection: bolt://localhost:7687
neo4j_user: neo4j
neo4j_password: ********
Neo4j Connection Successful


In [4]:
target_io_name = "COO-LowCoolantLevelGuard-PIN"

io = None
for io in io_list:
    if io["Name"] == target_io_name:
        break

io

{'Name': 'COO-LowCoolantLevelGuard-PIN',
 'NamePresentation': {'@edt': 'nfTxt', '#text': 'Coolant Level Warning Guard'},
 'PhysicalQuantity': {'@ref': 'PhysicalQuantity', '#text': 'Dimensionless'},
 'IOService': {'Name': 'COO-LowCoolantLevelGuard-PIN-readIO',
  'CommandType': 'readIO',
  'Description': {'@edt': 'fTxt'},
  'DiscreteVariable': {'@ref': 'DiscreteVariable', '#text': 'ScaniaState'}}}

In [5]:
ecu_system_execution = "C008"
for event in io_processor.stream(
            {
                "io_item": io,
                "ecu_system": ecu_system_execution,
                "base_configuration_components": [],
            },
            stream_mode="updates",
        ):
            print(event)
            if (
                "process_io_item" in event
                and event["process_io_item"]["matched"] == "yes"
            ):
                component_name = event["process_io_item"]["component"]

                print(f"Matched component: {component_name}")

{'process_io_item': {'matched': 'yes', 'component': 'R142'}}
Matched component: R142
